# Data Import

## SQL input query & Python joins, aggregations, and pivots for pre analysis dataframe formatting

In [ ]:
#Get pwd
#os.getcwd()
#Set current wd
#Connect to DB
import os
%run ./DigiSQLStartup.ipynb #Already imports libraries

#### Retrieve tags from History tbl

In [ ]:
%%sql tags <<
select * 
from MysteryMachine.tblAP1fcetags_edaexport
where ts>='2024-05-09'
order by ts asc

In [ ]:
dfTags = tags.DataFrame()
dfTags.info()
print(dfTags.shape)
dfTags.head(7)

#### Pivot float tag table tag_vals into pvt_tbl

In [ ]:
#Check if float vars have nan Value: Yes 2832
array=dfTags[~dfTags.Tagname.isin(["AP1_FurnaceCoilID","AP1_FurnaceThickness","AP1_FurnaceWidth"])].Value.isna()#.head(7)
#dfTags.where(array).dropna().head(7)
dfTags[~dfTags.Tagname.isin(["AP1_FurnaceCoilID","AP1_FurnaceThickness","AP1_FurnaceWidth"])].loc[array,:]
#array.sum()

In [ ]:
#Pivot tbl float tags
dfFloatTags = dfTags[~dfTags.Tagname.isin(["AP1_FurnaceCoilID","AP1_FurnaceThickness","AP1_FurnaceWidth"])].pivot(index='ts',columns='Tagname',values='Value')
print(dfFloatTags.shape)
dfFloatTags.head(3)

In [ ]:
type(dfFloatTags.index)

#### Pivot string tag table tag_vals into pvt_str_tbl

In [ ]:
#Pivot string tags tbl
dfStringTags=dfTags[dfTags.Tagname.isin(["AP1_FurnaceCoilID","AP1_FurnaceThickness","AP1_FurnaceWidth"])].pivot(index="ts",columns="Tagname",values="vValue")
print(dfStringTags.shape)
dfStringTags.head(3)

In [ ]:
type(dfStringTags.index)

#### Merge float tags and string tags


In [ ]:
#Merge float and string tbls
dfHistory = pd.merge(left=dfStringTags,right=dfFloatTags,how="inner",left_index=True,right_index=True)
print(dfHistory.shape)
dfHistory.AP1_FurnaceCoilID=dfHistory.AP1_FurnaceCoilID.str.upper()
dfHistory.head(3)

In [ ]:
type(dfHistory.index)

In [ ]:
#Check duplicated index
len(dfHistory.index[dfHistory.index.duplicated()])

##### Take the coils in dfHistory to extract info from tblProceso

In [ ]:
coils = tuple(dfHistory.AP1_FurnaceCoilID.drop_duplicates().to_list())
len(coils)#989

#### Add info tbl Proceso

In [ ]:
%%sql tblProceso <<
select top 2000000 LastProcessDate, CoilNumber, ProductID, ProductDivision, CurrentGuage, CurrentWidth, CoilLength, NetWeight, SteelGradeID
from database.schema.tblProceso tp
where  (LineID = 25) and ((tp.ProductID + COALESCE(tp.ProductDivision,'')) in :coils) 
order by LastProcessDate

In [ ]:
dfProceso = tblProceso.DataFrame()
dfProceso.set_index("LastProcessDate",inplace=True)
print(len(coils)) #989
print(dfProceso.CoilNumber.drop_duplicates().count())#917
dfProceso.head() #989, there are 72 coils in dfHistory that are not in dfProceso, because they haven't an entry with LineID=25
#Make upper() coil numbers
dfProceso.CoilNumber=dfProceso.CoilNumber.str.upper()
dfProceso.ProductID=dfProceso.ProductID.str.upper()
dfProceso.ProductDivision=dfProceso.ProductDivision.str.upper()
dfHistory["AP1_FurnaceCoilID"]=dfHistory["AP1_FurnaceCoilID"].str.upper()

In [ ]:
type(dfProceso.index)

#### Create input table X by merging dfHistory and dfProceso

In [ ]:
#Try solve group2 of the mismatched coils with direction="nearest" in merge_asof
dfXLong = pd.merge_asof(right=dfProceso,left=dfHistory,right_by="CoilNumber",left_by="AP1_FurnaceCoilID",right_index=True,left_index=True,direction="nearest",tolerance=pd.Timedelta("1D"))#
print(dfXLong.shape)
dfXLong.head(2)

In [ ]:
#Check duplicated index
len(dfXLong.index[dfXLong.index.duplicated()])

In [ ]:
#There're 1574 mistmaches between AP1_FurnaceCoilID and CoilNumber. Presumably bc it didnt find a time match
dfXMismatch = dfXLong[dfXLong.AP1_FurnaceCoilID != dfXLong.CoilNumber]
print(dfXMismatch.shape)
mismatchedCoilsX=list(dfXMismatch.AP1_FurnaceCoilID.unique())
print(len(mismatchedCoilsX))#Should be the 30 coil mismatches and the other 3 are time mismatches
dfXMismatch.head(2)

##### Remove mismatches from dfX

In [ ]:
print(mismatchedCoilsX)

In [ ]:
dfX=dfXLong[~dfXLong.AP1_FurnaceCoilID.isin(mismatchedCoilsX)]
print("Rows of X with mismatches: ", dfXLong.shape[0])
print("Rows of X without mismatches: ", dfX.shape[0])
print("Loss of {:.2f}% of rows due to mismatches".format((dfXLong.shape[0]-dfX.shape[0])/dfXLong.shape[0]*100))
print(dfX.shape)
dfX.head(2)

In [ ]:
#Check duplicated index
len(dfX.index[dfX.index.duplicated()])

In [ ]:
#Check there're no more mismatches
dfXMismatchNew = dfX[dfX.AP1_FurnaceCoilID != dfX.CoilNumber]
dfXMismatchNew.shape

In [ ]:
#Check no duplicated index:No as we've dropped mismatches
dfX.index[dfX.index.duplicated()]

#### Add pyro2goal

In [ ]:
%%sql pyro2goals <<
SELECT TOP (3000) [grade]
    ,[pyro2]
    ,[line]
    ,[Special]
    ,csg.SteelGradeID
FROM [database].[schema].[Pyro2goals] p2g
INNER JOIN database.schema.catSteelGrades csg ON CAST(p2g.grade AS VARCHAR(4))=csg.Name
WHERE line = 3310

In [ ]:
dfY = pyro2goals.DataFrame()
print(dfY.shape)
dfY.info()

In [ ]:
#Unique values in both columns, not much info apported, drop
print(dfY.line.unique())
print(dfY.Special.unique())

In [ ]:
#Drop 'line' and 'Special' from dfY
dfY.drop(["line","Special"],axis=1,inplace=True)
print(dfY.shape)
dfY.head(3)

In [ ]:
#Set SteelGradeID in dfX and dfY of same type
print(dfX.SteelGradeID.dtypes)
print(dfY.SteelGradeID.dtypes)
dfX=dfX.astype({'SteelGradeID': str(dfY.SteelGradeID.dtypes)})
dfX.SteelGradeID.dtypes

In [ ]:
#Check Duplicated: 6 rows are duplicated 
print("Rows of Y with duplicated values {}".format(dfY.shape[0]))
dfY.duplicated().sum()
dfY.loc[dfY.duplicated(),:]
dfY=dfY.drop_duplicates()
print("Rows of Y without duplicated values {}".format(dfY.shape[0]))
dfY.duplicated().sum()

In [ ]:
#Merge X & Y
dfXY = dfX.reset_index().merge(right=dfY,how="left",on="SteelGradeID").set_index("ts")
print(dfXY.shape)
dfXY.tail(2)

In [ ]:
#Check the right goal is assigned to each row: True
dfY.where(dfY.SteelGradeID==37).dropna()

In [ ]:
#Check no duplicated values: No
len(dfXY.index[dfXY.index.duplicated()])

## Variable import cleanup

In [ ]:
#AP1_FurnaceThickness,AP1_FurnaceWidth,CurrentGuage,CurrentWidth,CoilLength,NetWeight should be float64    
#AP1_FurnaceWeldInFurnace,AP1_FurnaceZone1TurndownOn,AP1_FurnaceZone2TurndownOn,AP1_FurnaceZone3TurndownOn,AP1_FurnaceZone4TurndownOn,AP1_FurnaceZone5TurndownOn,AP1_FurnaceZone6TurndownOn,AP1_FurnaceZone7TurndownOn,AP1_FurnaceZone8TurndownOn,grade should be categorical
#AP1_FurnaceCoilID,CoilNumber,ProductID,ProductDivision,SteelGradeID are of no use for the model
#dfXY.info(verbose=True)

In [ ]:
#Count number of coils 
dfXY.AP1_FurnaceCoilID.value_counts()#Counts how many times each unique coil value appears.
dfXY.AP1_FurnaceCoilID.drop_duplicates().count()#914 coils for our analysis

In [ ]:
#Drop unused columns
dfXY_coils = dfXY.AP1_FurnaceCoilID #we keep the indexed coil numbers in case
dfXY_coils.head()
dfXY.drop(["AP1_FurnaceCoilID","CoilNumber","ProductID","ProductDivision"],axis=1,inplace=True)
dfXY.info(verbose=True)

In [ ]:
dfXY_coils.head()

In [ ]:
#Check nan
dfXY.isna().sum()
#grade\pyro2 has 165 nan
#dfXY.grade.isna().sum()
#dfXY.pyro2.isna().sum()

In [ ]:
#Check SteelGradeID values for which pyro2 is nan
dfXY[dfXY.pyro2.isna()].SteelGradeID.sort_values().unique()#Nan values for 1,2,4 of steelgradeid

In [ ]:
#Check if for those grades all pyro2 values are nan: True
dfXY[dfXY.SteelGradeID.isin([1,2,4])][["SteelGradeID","pyro2"]].pyro2.notna().sum()

In [ ]:
#print("Unique SteelGradeID of Y",dfY.SteelGradeID.sort_values().unique())
#print("Unique SteelGradeID of X",dfX.SteelGradeID.sort_values().unique())
mismatchedSteelGradeID = [item for item in dfX.SteelGradeID.sort_values().unique() if item not in dfY.SteelGradeID.sort_values().unique()]
print("Unique SteelGradeID of X that are not in Y",mismatchedSteelGradeID)

In [ ]:
#Loss due to SteelGradeID mismatches
print("Rows in XY with mismatches in SteelGradeID: ",dfXY.shape[0])
print("Rows in XY without mismatches in SteelGradeID: ",dfXY[~dfXY.SteelGradeID.isin([1,2,4,97,109,238,290])].shape[0])
print("SteelGradeID mismatching rows in XY",dfXY[dfXY.SteelGradeID.isin([1,2,4,97,109,238,290])].shape[0])
print("Loss of {:.2f}% rows due to mismatches in SteelGradeID".format((dfXY[dfXY.SteelGradeID.isin([1,2,4,97,109,238,290])]).shape[0]/dfXY.shape[0]*100))

In [ ]:
dfXY[dfXY.SteelGradeID.isin([1,2,4])][["SteelGradeID","grade","pyro2"]].head(2)

In [ ]:
#Save mismatched tbl
dfXYmismatchPyro2 = dfXY.copy()

In [ ]:
#Set new goal to SteelGradeIDs 1,2,4:
dfXY.loc[dfXY[dfXY.SteelGradeID.isin([1,2,4])].index,["grade","pyro2"]]=dfXY[dfXY.SteelGradeID.isin([3,34])][["grade","pyro2"]].iloc[0].to_list()
dfXY[dfXY.SteelGradeID.isin([1,2,4])][["SteelGradeID","grade","pyro2"]].head()

In [ ]:
#Drop SteelGradeIDs 97,109,238,290 (mismatchedSteelGradeIDs that are not 1,2,4)
mismatchedSteelGradeIDNew = [item for item in mismatchedSteelGradeID if item not in [1,2,4]]
print("Loss of {:.2f}% rows due to mismatches in SteelGradeID".format((dfXY[dfXY.SteelGradeID.isin([97,109,238,290])]).shape[0]/dfXY.shape[0]*100))
dfXY = dfXY[~dfXY.SteelGradeID.isin([97,109,238,290])]

#Check SteelGradeIDs removed: True
dfXY[dfXY.SteelGradeID.isin(mismatchedSteelGradeID)].SteelGradeID.unique()
#Check no more nan: True
dfXY.grade.isna().sum()
dfXY.pyro2.isna().sum()

In [ ]:
#Change types
#Define floats
dfXY=dfXY.astype({'AP1_FurnaceThickness':'float64','AP1_FurnaceWidth':'float64','CurrentGuage':'float64','CurrentWidth':'float64','CoilLength':'float64','NetWeight': 'float64'})
#Define categorical first as int to remove decimals
dfXY[["AP1_FurnaceWeldInFurnace","AP1_FurnaceZone1TurndownOn","AP1_FurnaceZone2TurndownOn","AP1_FurnaceZone3TurndownOn","AP1_FurnaceZone4TurndownOn","AP1_FurnaceZone5TurndownOn","AP1_FurnaceZone6TurndownOn","AP1_FurnaceZone7TurndownOn","AP1_FurnaceZone8TurndownOn","grade","SteelGradeID"]]=dfXY[["AP1_FurnaceWeldInFurnace","AP1_FurnaceZone1TurndownOn","AP1_FurnaceZone2TurndownOn","AP1_FurnaceZone3TurndownOn","AP1_FurnaceZone4TurndownOn","AP1_FurnaceZone5TurndownOn","AP1_FurnaceZone6TurndownOn","AP1_FurnaceZone7TurndownOn","AP1_FurnaceZone8TurndownOn","grade","SteelGradeID"]].astype('int64')
#Define categorical
dfXY[["AP1_FurnaceWeldInFurnace","AP1_FurnaceZone1TurndownOn","AP1_FurnaceZone2TurndownOn","AP1_FurnaceZone3TurndownOn","AP1_FurnaceZone4TurndownOn","AP1_FurnaceZone5TurndownOn","AP1_FurnaceZone6TurndownOn","AP1_FurnaceZone7TurndownOn","AP1_FurnaceZone8TurndownOn","grade","SteelGradeID"]]=dfXY[["AP1_FurnaceWeldInFurnace","AP1_FurnaceZone1TurndownOn","AP1_FurnaceZone2TurndownOn","AP1_FurnaceZone3TurndownOn","AP1_FurnaceZone4TurndownOn","AP1_FurnaceZone5TurndownOn","AP1_FurnaceZone6TurndownOn","AP1_FurnaceZone7TurndownOn","AP1_FurnaceZone8TurndownOn","grade","SteelGradeID"]].astype("category")
dfXY.dtypes#info(verbose=True)

In [ ]:
#Check duplicated index
len(dfXY.index[dfXY.index.duplicated()])

In [ ]:
dfXY.isna().sum().sum()

### Adhoc graphing!